In [1]:
from leaderboard import *

## Alle Parts 2 (Sekunden)

In [2]:
data = load_data(leaderboard_id=976545, year=2024)
data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
3309362,238,6094,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
999754,321,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
attoPascal,303,2703,2189,1698,1558,36577,348,184,5433,13,3783,12901,125,3346,5995,251,14582,236
Belim,246,1864,839,928,1865,11227,742,654,9048,424,1668,7736,489,3119,6639,13566,8577,<NA>
Bene-dykt,1061,44036,382,1193,1777,4011,36294,928,3791,150,3042,33305,611,1003,<NA>,<NA>,<NA>,<NA>
berni664,494,4209,1856,1020,2395,8410,1107,1232,25898,543,5761,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
cluosh,<NA>,<NA>,411,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DavVader123,526,4812,903,2138,1949,4523,3618,5205,6111,197,6858,12106,175649,3251,70414,<NA>,<NA>,<NA>
emxchu,459,3418,29966,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
FelixSH,507,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## Part 2 Leaderboard

In [3]:
points(data).head(20)

,pts,name
1,698,Vonuras
2,646,msonnberger
3,644,JustAlittleWolf
4,644,Salominius
5,620,memoriesadrift
6,596,attoPascal
7,592,fwoerister
8,587,michalrzak
9,565,Belim
10,423,DavVader123


## Sekunden für aktuellen Tag, Part 2

In [4]:
seconds_latest_day(data)

,18
Vonuras,55
memoriesadrift,123
msonnberger,210
attoPascal,236
michalrzak,375
Salominius,646
fwoerister,1478


## Minuten für alle Parts 2

In [5]:
sum_seconds(data) // 60

,0
Vonuras,387
memoriesadrift,1152
Salominius,1501
attoPascal,1537
michalrzak,1612
fwoerister,1766
msonnberger,4905
